# Module 05: Windows Registry & System Configuration

**Difficulty**: ⭐⭐ (Intermediate)

**Estimated Time**: 60 minutes

**Prerequisites**: 
- Completed Modules 00-04
- Basic understanding of Windows system settings
- Administrator access recommended for some operations

## Learning Objectives

By the end of this notebook, you will be able to:

1. **Understand** the Windows Registry structure and purpose
2. **Read** registry values for system configuration
3. **Modify** environment variables permanently
4. **Automate** software configuration tasks
5. **Backup** and restore registry settings safely
6. **Apply** registry operations to data science workflows

## Introduction: Why Registry Matters for Data Scientists

The Windows Registry is a database that stores system and application settings. Understanding it allows you to:

### Common Use Cases

**1. Environment Configuration**
- Add directories to PATH permanently (not just current session)
- Set PYTHONPATH for custom modules
- Configure CUDA paths for GPU computing
- Set up proxy settings for corporate networks

**2. Software Automation**
- Configure installed software programmatically
- Set default file associations
- Automate application settings across machines
- Manage license keys securely

**3. System Discovery**
- Find installed software and versions
- Detect hardware configuration
- Check system capabilities
- Verify dependencies

**4. Team Standardization**
- Deploy consistent settings across team
- Automate new machine setup
- Share configuration scripts
- Document required settings

### ⚠️ Safety Warning

The Registry is powerful but dangerous:
- ❌ **Never** delete registry keys without understanding them
- ❌ **Never** modify HKEY_LOCAL_MACHINE without backup
- ✅ **Always** backup before changes
- ✅ **Always** test on non-production machine first
- ✅ **Always** use specific paths (not wildcards)

Incorrect registry changes can make Windows unbootable!

In [ ]:
# Setup: Import required libraries
import winreg  # Windows Registry access (Windows only)
import os
import sys
from pathlib import Path
import json

# Verify we're on Windows
import platform
if platform.system() != 'Windows':
    print("⚠ This module requires Windows")
    print("  Registry operations are Windows-specific")
else:
    print("✓ Running on Windows")
    print("Setup complete!")

## 1. Understanding the Registry Structure

The Registry is organized like a file system with keys (folders) and values (files).

### Root Keys (Hives)

| Key | Purpose | Common Use |
|-----|---------|------------|
| **HKEY_CURRENT_USER** (HKCU) | Current user settings | User preferences, paths |
| **HKEY_LOCAL_MACHINE** (HKLM) | System-wide settings | Installed software, hardware |
| **HKEY_CLASSES_ROOT** (HKCR) | File associations | Default programs |
| **HKEY_USERS** (HKU) | All user profiles | Multi-user settings |
| **HKEY_CURRENT_CONFIG** (HKCC) | Current hardware profile | Hardware settings |

### Value Types

- **REG_SZ**: String value
- **REG_DWORD**: 32-bit integer
- **REG_QWORD**: 64-bit integer
- **REG_EXPAND_SZ**: Expandable string (e.g., %USERPROFILE%)
- **REG_MULTI_SZ**: Array of strings
- **REG_BINARY**: Binary data

### Common Paths for Data Science

```
# User environment variables
HKCU\Environment

# System environment variables  
HKLM\SYSTEM\CurrentControlSet\Control\Session Manager\Environment

# Installed software (64-bit)
HKLM\SOFTWARE\Microsoft\Windows\CurrentVersion\Uninstall

# Python installations
HKCU\Software\Python
```

## 2. Reading Registry Values

Let's start with safe read-only operations.

In [ ]:
# Safe function to read registry value
# This is read-only and cannot harm your system

def read_registry_value(hive, path, value_name):
    """
    Read a single value from Windows Registry.
    
    Args:
        hive: Registry hive (e.g., winreg.HKEY_CURRENT_USER)
        path: Registry path (e.g., 'Environment')
        value_name: Name of value to read (e.g., 'PATH')
    
    Returns:
        tuple: (value, type) or (None, None) if not found
    """
    try:
        # Open registry key for reading
        key = winreg.OpenKey(hive, path, 0, winreg.KEY_READ)
        
        # Query value
        value, value_type = winreg.QueryValueEx(key, value_name)
        
        # Close key
        winreg.CloseKey(key)
        
        return value, value_type
    
    except FileNotFoundError:
        return None, None
    except PermissionError:
        print(f"Permission denied accessing registry key")
        return None, None

# Example: Read user's PATH variable
user_path, path_type = read_registry_value(
    winreg.HKEY_CURRENT_USER,
    'Environment',
    'PATH'
)

if user_path:
    print("User PATH variable:")
    print(f"Type: {path_type} (REG_SZ={winreg.REG_SZ}, REG_EXPAND_SZ={winreg.REG_EXPAND_SZ})")
    print(f"Value: {user_path[:100]}...")  # First 100 chars
else:
    print("User PATH not set in registry")

### 2.1 Discovering Installed Software

In [ ]:
# Find installed software from Registry
# Useful for checking if dependencies are installed

def get_installed_software():
    """
    Get list of installed software from Registry.
    
    Returns:
        list: List of dicts with software info
    """
    software_list = []
    
    # Check both 64-bit and 32-bit locations
    paths = [
        r"SOFTWARE\Microsoft\Windows\CurrentVersion\Uninstall",
        r"SOFTWARE\WOW6432Node\Microsoft\Windows\CurrentVersion\Uninstall"
    ]
    
    for path in paths:
        try:
            # Open uninstall key
            key = winreg.OpenKey(winreg.HKEY_LOCAL_MACHINE, path, 0, winreg.KEY_READ)
            
            # Enumerate all subkeys (each is an installed program)
            i = 0
            while True:
                try:
                    subkey_name = winreg.EnumKey(key, i)
                    subkey = winreg.OpenKey(key, subkey_name)
                    
                    # Try to get DisplayName
                    try:
                        name, _ = winreg.QueryValueEx(subkey, 'DisplayName')
                        version, _ = winreg.QueryValueEx(subkey, 'DisplayVersion')
                        
                        software_list.append({
                            'name': name,
                            'version': version
                        })
                    except FileNotFoundError:
                        pass
                    
                    winreg.CloseKey(subkey)
                    i += 1
                    
                except OSError:
                    break
            
            winreg.CloseKey(key)
        
        except PermissionError:
            pass
    
    return software_list

# Example: Find Python installations
all_software = get_installed_software()
python_software = [s for s in all_software if 'python' in s['name'].lower()]

print(f"Found {len(all_software)} installed programs")
print(f"\nPython-related software ({len(python_software)}):")
for software in python_software[:5]:
    print(f"  - {software['name']} (v{software['version']})")

## 3. Managing Environment Variables

The most common registry operation for data scientists: managing PATH and other environment variables.

### Why Use Registry for Environment Variables?

**Temporary (current session only)**:
```python
os.environ['MY_VAR'] = 'value'  # Lost when session ends
```

**Permanent (survives restarts)**:
```python
# Set in Registry → Available in all future sessions
```

### Important Notes

- User variables: `HKCU\Environment` (no admin needed)
- System variables: `HKLM\SYSTEM\...\Environment` (requires admin)
- Changes require new terminal to take effect
- Use `setx` command as simpler alternative when possible

In [ ]:
# Read current user environment variables
# Safe read-only operation

def get_user_environment_variables():
    """
    Get all user environment variables from Registry.
    
    Returns:
        dict: Environment variables and their values
    """
    env_vars = {}
    
    try:
        key = winreg.OpenKey(winreg.HKEY_CURRENT_USER, 'Environment', 0, winreg.KEY_READ)
        
        i = 0
        while True:
            try:
                name, value, value_type = winreg.EnumValue(key, i)
                env_vars[name] = value
                i += 1
            except OSError:
                break
        
        winreg.CloseKey(key)
    
    except PermissionError:
        pass
    
    return env_vars

# Display user environment variables
user_env = get_user_environment_variables()

print(f"User environment variables ({len(user_env)}):")
print("=" * 70)
for name, value in sorted(user_env.items())[:10]:  # Show first 10
    value_str = str(value)[:60] + '...' if len(str(value)) > 60 else str(value)
    print(f"{name}: {value_str}")

if len(user_env) > 10:
    print(f"... and {len(user_env) - 10} more")

### 3.1 Adding to PATH (Using setx - Recommended)

For PATH modification, the `setx` command is safer and simpler than direct registry editing.

In [ ]:
# Add directory to user PATH using setx command
# This is the recommended method (safer than direct registry editing)

import subprocess

def add_to_user_path(directory, dry_run=True):
    """
    Add a directory to user PATH variable.
    
    Args:
        directory: Directory path to add
        dry_run: If True, only show what would be done
    
    Returns:
        bool: Success status
    """
    directory = str(Path(directory).resolve())
    
    # Get current PATH
    current_path, _ = read_registry_value(
        winreg.HKEY_CURRENT_USER,
        'Environment',
        'PATH'
    )
    
    if current_path is None:
        current_path = ''
    
    # Check if already in PATH
    path_dirs = [p.strip() for p in current_path.split(';') if p.strip()]
    
    if directory in path_dirs:
        print(f"✓ {directory} already in PATH")
        return True
    
    # Add to PATH
    new_path = current_path + ';' + directory if current_path else directory
    
    print(f"{'DRY RUN: Would add' if dry_run else 'Adding'} to PATH:")
    print(f"  {directory}")
    
    if not dry_run:
        try:
            # Use setx command to set PATH
            result = subprocess.run(
                ['setx', 'PATH', new_path],
                capture_output=True,
                text=True
            )
            
            if result.returncode == 0:
                print("✓ PATH updated successfully")
                print("  Note: Restart terminal for changes to take effect")
                return True
            else:
                print(f"✗ Failed: {result.stderr}")
                return False
        
        except Exception as e:
            print(f"✗ Error: {e}")
            return False
    else:
        print("\nSet dry_run=False to actually modify PATH")
        return True

# Example: Add a custom scripts directory (dry run)
# add_to_user_path(r'C:\Users\YourName\scripts', dry_run=True)
print("add_to_user_path() function ready!")
print("⚠ Use with caution - always test with dry_run=True first")

## 4. Practice Exercises

### Exercise 1: Software Version Checker

Create a function that checks if required software is installed:
1. Accept list of software names
2. Search Registry for each
3. Report found vs not found
4. Check version requirements

**Hint**: Use `get_installed_software()` and version comparison

In [ ]:
# Exercise 1: Your solution here

def check_software_requirements(requirements):
    """
    Check if required software is installed.
    
    Args:
        requirements: List of (name, min_version) tuples
    
    Returns:
        dict: Status for each requirement
    """
    # TODO: Implement this function
    pass

# Test your function
# requirements = [('Python', '3.8'), ('Git', '2.0')]
# check_software_requirements(requirements)


### Exercise 2: Environment Variable Manager

Create a tool to manage user environment variables:
1. List all current variables
2. Search for specific variable
3. Suggest cleanup (unused variables)
4. Export to JSON for backup

**Hint**: Use `get_user_environment_variables()` as starting point

In [ ]:
# Exercise 2: Your solution here

class EnvironmentVariableManager:
    """
    Manage user environment variables.
    """
    
    def list_all(self):
        # TODO: List all variables
        pass
    
    def search(self, pattern):
        # TODO: Search for variables matching pattern
        pass
    
    def export_to_json(self, filename):
        # TODO: Export to JSON file
        pass

# Test your manager
# manager = EnvironmentVariableManager()
# manager.list_all()


### Exercise 3: PATH Analyzer

Create a PATH analysis tool:
1. List all directories in PATH
2. Check which ones exist vs missing
3. Find duplicate entries
4. Suggest cleanup
5. Generate cleaned PATH

**Hint**: Split PATH by `;` and check with `Path().exists()`

In [ ]:
# Exercise 3: Your solution here

def analyze_path():
    """
    Analyze PATH variable and suggest improvements.
    
    Returns:
        dict: Analysis results
    """
    # TODO: Implement PATH analysis
    pass

# Test your analyzer
# analyze_path()


## 5. Summary

### Key Concepts

1. **Registry Structure**
   - Organized as hives (HKCU, HKLM, etc.)
   - Keys (like folders) and Values (like files)
   - Different value types (REG_SZ, REG_DWORD, etc.)

2. **Reading Registry**
   - `winreg.OpenKey()` to access keys
   - `winreg.QueryValueEx()` to read values
   - Always close keys after use
   - Handle exceptions gracefully

3. **Environment Variables**
   - User: `HKCU\Environment` (no admin needed)
   - System: `HKLM\SYSTEM\...\Environment` (admin required)
   - Use `setx` for PATH modifications
   - Restart terminal for changes to apply

4. **Installed Software**
   - Check `HKLM\SOFTWARE\...\Uninstall`
   - Check both 64-bit and 32-bit locations
   - Useful for dependency verification

5. **Safety Practices**
   - Always backup before modifications
   - Test on non-production first
   - Use specific paths (no wildcards)
   - Prefer `setx` over direct registry edits

### Real-World Applications

- **Environment Setup**: Add directories to PATH permanently
- **Dependency Check**: Verify required software installed
- **Configuration**: Standardize settings across team
- **Automation**: Script machine setup

### What's Next?

In **Module 06: Networking Basics**, you'll learn:
- Network configuration and diagnostics
- HTTP requests and APIs
- Port testing and connectivity
- Web scraping considerations

### Self-Assessment

Before moving on, make sure you can:
- [ ] Read registry values safely
- [ ] Find installed software
- [ ] Manage user environment variables
- [ ] Add directories to PATH
- [ ] Understand registry safety practices

---

**Continue to Module 06** when ready!